In [1]:
from sage.geometry.hyperbolic_space.hyperbolic_model import moebius_transform
UHP = HyperbolicPlane().UHP()
from fractions import Fraction
import itertools

margin_of_error = 0.000001

#
#general functions
#

def short_str(number):
    return str(number.n(digits=10))

def short_str_list(list_):
    short_list = []
    for elem in list_:
        short_list.append(short_str(elem))
    return str(short_list)

def normOfDifference(X,Y):
    norm = 0
    for i in range(len(X)):
        norm = norm + abs(X[i]-Y[i])
    return norm

#https://github.com/python/cpython/issues/95723
def limit_denominator(m: int, n: int, l: int) -> tuple[int, int]:
    a, b, p, q, r, s, v = n, m % n, 1, 0, m // n, 1, 1
    while 0 < b and q + a // b * s <= l:
        a, b, p, q, r, s, v = b, a % b, r, s, p + a // b * r, q + a // b * s, -v
    t, u = p + (l - q) // s * r, q + (l - q) // s * s
    return (r, s) if 2 * b * u <= n else (t, u)

def anglesToFractions(decimals):
    fractions = []
    for number in decimals:
        approx = Fraction(float(number/pi))
        best_approx = limit_denominator(approx.numerator, approx.denominator, 100)
        if abs(best_approx[0]/best_approx[1]*pi-number) < margin_of_error*margin_of_error:
            if best_approx[0] == 1 and best_approx[1] == 1:
                fractions.append("pi")
            elif best_approx[0] == 1:
                fractions.append("pi/"+ str(best_approx[1]))
            elif best_approx[0] == 0:
                fractions.append("0")
            else:
                fractions.append(str(best_approx[0]) + "pi/"+ str(best_approx[1]))
        else:
            fractions.append(str(number))
    return fractions

#
#hyperbolic functions
#

def mobius(matrix_, point):
    return UHP.get_point(moebius_transform(matrix_,point.coordinates()))

def floatReal(point):
    return float(point.coordinates().real())

def floatImag(point):
    return float(point.coordinates().imag())

def floatPoint(point):
    return UHP.get_point(floatReal(point) + I*floatImag(point))

def inverse(matrix_):
    a = matrix_[0][0]
    b = matrix_[0][1]
    c = matrix_[1][0]
    d = matrix_[1][1]
    
    return matrix(CDF, 2, [d, -b, -c, a])

def elliptic(point, angle):
    x = floatReal(point)
    y = floatImag(point)
    return matrix(CDF, 2, [cos(angle/2)-x*sin(angle/2)/y,((1/y)*(x^2)+y)*sin(angle/2),-sin(angle/2)/y,cos(angle/2)+x*sin(angle/2)/y])
    #see paper for formula of the matrix given fixed point ang angle, anti-clockwise rotation !!

def angleElliptic(matrix_):
    a = matrix_[0][0]
    b = matrix_[0][1]
    c = matrix_[1][0]
    d = matrix_[1][1]

    sign = -c/abs(c)
    square_trace = (a+d)^2

    if abs(a+d) >= 2:
        print("Not Elliptic!")
    else:
        angle = atan(sign*(a+d)/(square_trace-2)*sqrt(4-square_trace))

        if square_trace > 2 and (a+d)*sign > 0:
            return angle
        elif square_trace > 2 and (a+d)*sign < 0:
            return angle + 2*pi
        else:
            return angle + pi

def fixedPointElliptic(matrix_):
    a = matrix_[0][0]
    b = matrix_[0][1]
    c = matrix_[1][0]
    d = matrix_[1][1]
    
    return UHP.get_point((a-d)/(2*c) + I*sqrt(4-(a+d)*(a+d))/(2*sqrt(c*c)))

def inclinationRay(X,A):
    ray = HyperbolicPlane().UHP().get_geodesic(X, A)
    vertical = HyperbolicPlane().UHP().get_geodesic(X, infinity)
    center = X.coordinates().real()
    radius = X.coordinates().imag()

    A_real = A.coordinates().real()
    A_imag = A.coordinates().imag()
    
    #case X and A are on the same vertical geodesic
    if abs(A_real-center) < margin_of_error:
        if A_imag > radius:
            return 0
        else:
            return pi
    #general case
    elif (A_real-center)^2 + A_imag^2 > radius^2 and A_real < center:
        return vertical.angle(ray)
    elif (A_real-center)^2 + A_imag^2 < radius^2 and A_real < center:
        return pi-vertical.angle(ray)
    elif (A_real-center)^2 + A_imag^2 < radius^2 and A_real > center:
        return pi+vertical.angle(ray)
    else:
        return 2*pi-vertical.angle(ray)

def angleBetweenTwoRays(X,A,B):
    return inclinationRay(X,B)-inclinationRay(X,A)

class triangle:

    def __init__(self, angle1, angle2, angle3, vertex1, inclination):
        self.angles = [angle1, angle2, angle3]
        self.vertex1 = vertex1
        self.inclination = inclination

    def area(self):
        return pi - sum(self.angles)

    def getLengths(self):
        angle1 = self.angles[0]
        angle2 = self.angles[1]
        angle3 = self.angles[2]

        length1 = float(acosh((cos(angle1)+cos(angle2)*cos(angle3))/(sin(angle2)*sin(angle3))))
        length2 = float(acosh((cos(angle2)+cos(angle1)*cos(angle3))/(sin(angle1)*sin(angle3))))
        length3 = float(acosh((cos(angle3)+cos(angle1)*cos(angle2))/(sin(angle1)*sin(angle2))))
        #see hyperbolic law of cosines

        return [length1, length2, length3]

    def getVertex2(self):
        if sum(self.angles) > pi - margin_of_error:
            return floatPoint(self.vertex1)
        else:
            vertex1 = self.vertex1
            vertex1_real = floatReal(vertex1)
            vertex1_imag = floatImag(vertex1)
            matrix_ = elliptic(vertex1, -self.inclination)
            #minus inclination because we want to rotate clockwise
            length3 = self.getLengths()[2]


            vertex2 = mobius(matrix_, UHP.get_point(vertex1_real + I*((vertex1_imag)*exp(length3))))
            #rotate a point on the complex line, at hyperbolic distance length3 of vertex1
            return floatPoint(vertex2)

    def getVertex3(self):
        if sum(self.angles) > pi - margin_of_error:
            return floatPoint(self.vertex1)
        else:
            vertex1 = self.vertex1
            vertex1_real = floatReal(vertex1)
            vertex1_imag = floatImag(vertex1)
            length2 = self.getLengths()[1]

            matrix_ = elliptic(vertex1, -self.inclination-self.angles[0])


            vertex3 = mobius(matrix_, UHP.get_point(vertex1_real + I*((vertex1_imag)*exp(length2))))

            #idem, but length2
            return floatPoint(vertex3)

    def getSides(self):
        vertex1 = self.vertex1
        vertex2 = self.getVertex2()
        vertex3 = self.getVertex3()

        side1 = HyperbolicPlane().UHP().get_geodesic(vertex2, vertex3)
        side2 = HyperbolicPlane().UHP().get_geodesic(vertex1, vertex3)
        side3 = HyperbolicPlane().UHP().get_geodesic(vertex1, vertex2)
        return [side1, side2, side3]

    def getInclinationThirdVertex(self):
        return inclinationRay(self.getVertex3(), self.getVertex2())

    def getSidesGraphics(self):
        sides = self.getSides()
        picture = Graphics()
        for side in sides:
            picture += side.plot(False, color='black')
        return picture

    def draw(self):
        show(self.getSidesGraphics())

class chain_of_triangles:

    def __init__(self, alpha, beta, gamma, basepoint, initial_inclination):
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.initial_inclination = initial_inclination
        self.basepoint = basepoint

        self.sanityCheck()

    def numberOfPunctures(self):
        return len(self.alpha)

    def numberOfTriangles(self):
        return self.numberOfPunctures()-2

    def checkLengthOfEntries(self):
        if not(len(self.alpha) == len(self.beta)+3 and len(self.beta) == len(self.gamma)):
            print("Problem with the length of entries :(")

    def checkAnglesCondition(self):
        if sum(self.alpha) <= 2*pi*(len(self.alpha)-1):
            print("Angles condition failed :(")

        no_error_range_beta = True

        if len(self.beta) != 0:

            if self.beta[0] < 4*pi-self.alpha[0]-self.alpha[1]:
                print("Angle beta_" + str(1) +" out of range, it should be at least 2pi*(" + str(round((4*pi-self.alpha[0]-self.alpha[1])/(2*pi),3)) + ")")
                no_error_range_beta = False

            for i in range(len(self.beta)-1):

                upper_bound = self.alpha[i+2]-2*pi

                if self.beta[i] - self.beta[i+1] > upper_bound:
                    print("Angle beta_" + str(i+2) +" out of range, it should be at least 2pi*(" + str(round((self.beta[i]-upper_bound)/(2*pi),3)) + ")")
                    no_error_range_beta = False

            if self.beta[len(self.beta)-1] > self.alpha[len(self.alpha)-2]+self.alpha[len(self.alpha)-1]-2*pi:
                print("Angle beta_" + str(len(self.beta)) +" out of range, it should be at most 2pi*(" + str(round((self.alpha[len(self.alpha)-2]+self.alpha[len(self.alpha)-1]-2*pi)/(2*pi),3)) + ")")
                no_error_range_beta = False

        #if no_error_range_beta:
            #print ("All beta_i whithin range :)")

    def sanityCheck(self):
        self.checkLengthOfEntries()
        self.checkAnglesCondition()

    def alphaAngles(self):
        angle = []
        for ele in self.alpha:
            angle.append(pi - ele/2)
        return angle

    def getTriangles(self):
        triangles=[]
        numberOfTriangles = self.numberOfTriangles()
        numberOfPunctures = self.numberOfPunctures()
        alphaAngles = self.alphaAngles()
        beta = self.beta
        gamma = self.gamma

        #special case of only one triangle
        #print(numberOfTriangles, " triangles")
        if numberOfTriangles == 1:
            Delta_0 = triangle(alphaAngles[0],alphaAngles[1],alphaAngles[2], self.basepoint, self.initial_inclination)
            triangles.append(Delta_0)
        else:
            #first set up the first triangle Delta_0
            Delta_0 = triangle(alphaAngles[0],alphaAngles[1],pi-beta[0]/2, self.basepoint, self.initial_inclination)
            triangles.append(Delta_0)

            #now set up all but the last triangle
            #note that range(x,y) are the numbers x, x+1,..., y-1
            for i in range(1, numberOfTriangles-1):
                Delta_previous = triangles[i-1]
                B_actual = Delta_previous.getVertex3()
                inclination = Delta_previous.getInclinationThirdVertex()
                Delta_actual = triangle(beta[i-1]/2,alphaAngles[i+1],pi-beta[i]/2, B_actual, gamma[i-1]-inclination)
                triangles.append(Delta_actual)

            #now set up the last triangle
            Delta_penultimate = triangles[numberOfTriangles-2]
            B_final = Delta_penultimate.getVertex3()
            inclination_penultimate = Delta_penultimate.getInclinationThirdVertex()

            Delta_final = triangle(beta[numberOfTriangles-2]/2,alphaAngles[numberOfPunctures-2],alphaAngles[numberOfPunctures-1], B_final, gamma[numberOfTriangles-2]-inclination_penultimate)
            triangles.append(Delta_final)

        return triangles

    def getVerticesC(self):
        triangles = self.getTriangles()
        numberOfTriangles = self.numberOfTriangles()

        Delta_0 = triangles[0]
        verticesC = [Delta_0.vertex1, Delta_0.getVertex2(), Delta_0.getVertex3()]

        count = 1

        while count < numberOfTriangles:
            verticesC.pop()
            Delta_i = triangles[count]
            verticesC.append(Delta_i.getVertex2())
            verticesC.append(Delta_i.getVertex3())
            count += 1

        return verticesC

    def getRepresentation(self):
        verticesC = self.getVerticesC()
        alpha = self.alpha

        matrices = []

        for i in range(len(verticesC)):
            matrix_ = elliptic(verticesC[i],alpha[i])
            matrices.append(matrix_)

        return representation(matrices)

    def printRepresentation(self, only_beta_gamma = False, image_of_generators = False, volume_check = False, angle_of_simple_closed_curves = False):
        matrices = self.getRepresentation().images
        product = matrix(2,[1,0,0,1])
        alpha = self.alpha
        
        #all_angles = "alpha = " + short_str_list(self.alpha) + ", beta = " + short_str_list(self.beta) + ", gamma = " + str(self.gamma)
        all_angles_fractions = "alpha = " + str(anglesToFractions(self.alpha)) + ", beta = " + str(anglesToFractions(self.beta)) + ", gamma = " + str(anglesToFractions(self.gamma))
        
        if only_beta_gamma:
            #all_angles = "beta = " + short_str_list(self.beta) + ", gamma = " + str(self.gamma)
            all_angles_fractions = "beta = " + str(anglesToFractions(self.beta)) + ", gamma = " + str(anglesToFractions(self.gamma))
        
        if not(only_beta_gamma):
            print("Representation of a " + str(self.numberOfPunctures()) + "-punctured sphere with")
        #print(all_angles)
        print(all_angles_fractions)
        print()
        
        if image_of_generators:
            for i in range(len(matrices)):
                print("Image of generator " + str(i+1) + " is the matrix\n" + matrices[i].str(rep_mapping=short_str))
                print("The PSL(2,R) angle of rotation is " + short_str(angleElliptic(matrices[i])/(2*pi)) + "*2pi and alpha_" + str(i+1) + " is " + short_str(alpha[i]/(2*pi)) +"*2pi.")
                print()
                product = product * matrices[i]
            print("The product of the images of the generators is\n" + product.str(rep_mapping=short_str))
            print()
        
        if volume_check:
            triangles = self.getTriangles()
            area = 0
            for triangle in triangles:
                area += triangle.area()

            print("Volume check: Twice the total area is " + short_str(-2*area) + " and the scaling factor (expected volume) is " + short_str(2*pi*(self.numberOfPunctures()-1)-sum(alpha)))

        if angle_of_simple_closed_curves:
            print()
            print("Image of b_1 is the matrix\n" + (inverse(matrices[0]*matrices[1])).str(rep_mapping=short_str))
            print("The PSL(2,R) angle of rotation of b_1 is " + short_str(angleElliptic(inverse(matrices[0]*matrices[1]))/(2*pi)) + "*2pi.")
            print()
        
            print()
            print("Image of d_1 is the matrix\n" + (inverse(matrices[1]*matrices[2])).str(rep_mapping=short_str))
            print("The PSL(2,R) angle of rotation of d_1 is " + short_str(angleElliptic(inverse(matrices[1]*matrices[2]))/(2*pi)) + "*2pi.")
            print()
            
            print()
            print("Image of e_1 is the matrix\n" + (inverse(matrices[1]*matrices[3])).str(rep_mapping=short_str))
            print("The PSL(2,R) angle of rotation of e_1 is " + short_str(angleElliptic(inverse(matrices[1]*matrices[3]))/(2*pi)) + "*2pi.")
            print()

    def getGraphics(self):
        picture = Graphics()
        triangles = self.getTriangles()
        for triangle in triangles:
            picture += triangle.getSidesGraphics()
        return picture

    def draw(self):
        show(self.getGraphics())
        
def generatorsMCG(number_of_punctures):
        generators = []
        for i in range(number_of_punctures-2):
            curve = [i+1]
            new_generators = []
            for j in range(i+1, number_of_punctures-1):
                if j-i < number_of_punctures-2:
                    curve = curve + [j+1]
                    new_generators += [curve]
            generators = generators + new_generators
        return generators
    
class representation:
    
    def __init__(self, images):
        self.images = images
    
    def numberOfPunctures(self):
        return len(self.images)
    
    def alphaAngles(self):
        alpha=[]
        for matrix_ in self.images:
            alpha.append(angleElliptic(matrix_))
        return alpha
    
    #curve is given as a list [i, i+1, ..., j] and by convention the image of the curve is the image of (c_ic_i+1...c_j)^-1   
    def imageCurve(self, curve):
        image_of_curve = matrix(2,[1,0,0,1])
        for index in curve:
            image_of_curve = image_of_curve*self.images[index-1]
        return inverse(image_of_curve)
    
    def angleCurve(self, curve):
        return angleElliptic(self.imageCurve(curve))
    
    def imageCurves_B(self):
        curve=[1]
        image_B=[]
        for i in range(len(self.images)-3):
            curve.append(i+2)
            image_B.append(self.imageCurve(curve))
        return image_B
    
    def betaAngles(self):
        beta=[]
        for elem in self.imageCurves_B():
            beta.append(angleElliptic(elem))
        return beta
    
    def getVerticesC(self):
        verticesC=[]
        for elem in self.images:
            verticesC.append(fixedPointElliptic(elem))
        return verticesC
    
    def getVerticesB(self):
        verticesB=[]
        for elem in self.imageCurves_B():
            verticesB.append(fixedPointElliptic(elem))
        return verticesB
    
    def gammaAngles(self):
        verticesC=self.getVerticesC()
        verticesB=self.getVerticesB()
        gamma=[]
        
        for i in range(len(verticesB)):         
            if UHP.dist(verticesB[i], verticesC[i+2]) < margin_of_error:
                gamma.append(0)
            else:
                j=i+1
                while UHP.dist(verticesB[i], verticesC[j]) < margin_of_error and j>0:
                    j-=1
                    
                if j==0:
                    gamma.append(0)
                else:
                    angle = angleBetweenTwoRays(verticesB[i], verticesC[i+2], verticesC[j])
                    while angle < 0:
                        angle = angle + 2*pi
                    while angle > 2*pi - margin_of_error:
                        angle = angle - 2*pi
                    gamma.append(float(angle))

        return gamma

    def getTriangleChain(self):
        return chain_of_triangles(self.alphaAngles(), self.betaAngles(), self.gammaAngles(), UHP.get_point(I), 0)

    def ImageByDehnTwist(self, curve, iterations = 1): #curve is of the form c_ic_i+1...c_j, given by the two indices [i,j]    
        image_of_curve = self.imageCurve(curve)
        image_of_curve_iterated = image_of_curve
        for i in range(iterations-1):
            image_of_curve_iterated = image_of_curve_iterated * image_of_curve

        new_representation = []
        for i in range(len(self.images)):
            if i+1 in curve:
                new_representation.append(image_of_curve_iterated*(self.images[i])*inverse(image_of_curve_iterated))
            else:
                new_representation.append(self.images[i])

        return representation(new_representation)

    def printImageByGenerators(self):
        number_of_punctures = self.numberOfPunctures()

        for curve in generatorsMCG(number_of_punctures):
            angle = self.angleCurve(curve)
            angle_approx = Fraction(float(angle/(2*pi)))
            angle_best_approx = limit_denominator(angle_approx.numerator, angle_approx.denominator, 100)
            order = angle_best_approx[1]

            print("Image(s) by twist along the curve " + str(curve))
            print()
            for i in range(order-1):
                print("Iteration " + str(i+1))
                self.ImageByDehnTwist(curve, iterations = i+1).getTriangleChain().printRepresentation(image_of_generators = False, volume_check = False, angle_of_simple_closed_curves = False)
    
    def getPartialOrbit(self, depth):
        number_of_punctures = self.numberOfPunctures()
        generators = generatorsMCG(number_of_punctures)
        
        #set the base point
        initial_orbit_point = orbit_point(self, list())
        orbit_ = orbit([initial_orbit_point])
        
        current_depth = 1

        while current_depth <= depth:
            new_orbit_points = []

            #consider all orbit points of longest depth
            for orbit_elem in orbit_.orbit_points:
                if orbit_elem.depth() == current_depth - 1:
                    #apply each generator to these elements
                    for elem in generators:
                        new_orbit_point = orbit_point(orbit_elem.getRepresentation().ImageByDehnTwist(elem), orbit_elem.getGroupElement() + [elem])
                        #check it's indeed a new element
                        already_counted = False
                        for orbit_elem_ in orbit_.orbit_points + new_orbit_points:
                            if normOfDifference(new_orbit_point.representation.betaAngles(), orbit_elem_.representation.betaAngles()) < margin_of_error and normOfDifference(new_orbit_point.representation.gammaAngles(), orbit_elem_.representation.gammaAngles()) < margin_of_error:
                                already_counted = True
                        if not(already_counted):
                            new_orbit_points.append(new_orbit_point)

            #add the new orbit points created
            orbit_.add(new_orbit_points)
            current_depth = current_depth + 1
        return orbit_

    def printPartialOrbit(self, depth, only_beta_gamma = True, ordered_by_distance_to_basepoint = True, ordered_by_beta = True):
        orbit_ = self.getPartialOrbit(depth)
        
        print("Length of orbit up to depth " + str(depth) + " is " + str(orbit_.length()))
        print()
        
        if ordered_by_distance_to_basepoint:
            print("List of orbit points ordered by distance from the basepoint:")
            print()
            orbit_.print_(only_beta_gamma)
        
        if ordered_by_beta:
            print("List of orbit points ordered by beta:")
            print()
            orbit_.orderByBeta().print_(only_beta_gamma)
            
class orbit_point:
    
    def __init__(self, representation, group_element):
        self.representation = representation
        self.group_element = group_element
        
    def depth(self):
        return len(self.group_element)
    
    def getRepresentation(self):
        return self.representation
    
    def getGroupElement(self):
        return self.group_element

class orbit:
    
    def __init__(self, orbit_points):
        self.orbit_points = orbit_points
    
    def length(self):
        return len(self.orbit_points)
    
    def add(self, new_orbit_points):
        self.orbit_points += new_orbit_points
    
    def print_(self, only_beta_gamma = True):
        for elem in self.orbit_points:
            print("Image by " + str(elem.group_element))
            elem.representation.getTriangleChain().printRepresentation(only_beta_gamma, image_of_generators = False, volume_check = False, angle_of_simple_closed_curves = False)
    
    def getAllBetaAngles(self):
        beta_angles = []
        for orbit_point in self.orbit_points:
            new_beta_angles = orbit_point.getRepresentation().betaAngles()
            
            already_counted = False
            for elem in beta_angles:
                if normOfDifference(elem, new_beta_angles) < margin_of_error:
                    already_counted = True
            if not(already_counted):
                beta_angles.append(new_beta_angles)
        
        return beta_angles  
    
    def orderByBeta(self):
        beta_angles = self.getAllBetaAngles()
        
        for beta in beta_angles:
            for i in range(len(self.orbit_points)):
                beta_to_compare = self.orbit_points[i].getRepresentation().betaAngles()
                if normOfDifference(beta, beta_to_compare) < margin_of_error:
                    self.orbit_points.insert(0, self.orbit_points.pop(i))
        
        self.orbit_points.reverse()
        return self
    

In [8]:
#enter angle vector alpha of the DT component 
alpha = [12*pi/7, 12*pi/7, 12*pi/7, 12*pi/7, 12*pi/7, 12*pi/7]

#start with a point, given in action-angle coordinates
beta = [2*pi/3, pi, 4*pi/3]
gamma = [2*pi/3, 0, 2*pi/3]

#set up initial triangle chain
myChain = chain_of_triangles(alpha, beta, gamma, UHP.get_point(I), 5*pi/14)

#draw the cahin
#myChainFig = myChain.getGraphics()
#show(myChainFig)

#print image of generators as 2x2 matrices, with other options
#myChain.printRepresentation(image_of_generators = True, volume_check = False, angle_of_simple_closed_curves = True)

#compute orbit points to a certain depth
myChain.getRepresentation().printPartialOrbit(depth = 6, only_beta_gamma = True, ordered_by_distance_to_basepoint = True, ordered_by_beta = True)
myChain.getRepresentation().getPartialOrbit(3).getAllBetaAngles()



Length of orbit up to depth 6 is 40

List of orbit points ordered by distance from the basepoint:

Image by []
beta = ['2pi/3', 'pi', '4pi/3'], gamma = ['2pi/3', '0', '2pi/3']

Image by [[1, 2]]
beta = ['2pi/3', 'pi', '4pi/3'], gamma = ['4pi/3', '0', '2pi/3']

Image by [[1, 2, 3]]
beta = ['2pi/3', 'pi', '4pi/3'], gamma = ['2pi/3', 'pi', '2pi/3']

Image by [[1, 2, 3, 4]]
beta = ['2pi/3', 'pi', '4pi/3'], gamma = ['2pi/3', '0', '0']

Image by [[2, 3]]
beta = ['4pi/7', 'pi', '4pi/3'], gamma = ['0', 'pi/2', '2pi/3']

Image by [[2, 3, 4]]
beta = ['2pi/3', 'pi', '4pi/3'], gamma = ['4pi/3', '0', '4pi/3']

Image by [[4, 5]]
beta = ['2pi/3', 'pi', '4pi/3'], gamma = ['2pi/3', 'pi', '4pi/3']

Image by [[1, 2], [1, 2]]
beta = ['2pi/3', 'pi', '4pi/3'], gamma = ['0', '0', '2pi/3']

Image by [[1, 2], [1, 2, 3]]
beta = ['2pi/3', 'pi', '4pi/3'], gamma = ['4pi/3', 'pi', '2pi/3']

Image by [[1, 2], [1, 2, 3, 4]]
beta = ['2pi/3', 'pi', '4pi/3'], gamma = ['4pi/3', '0', '0']

Image by [[1, 2], [2, 3, 4, 5]]


[[pi - 1.0471975511965974,
  pi + 2.220446049250313e-16,
  pi + 1.0471975511965983],
 [pi - 1.346396851538483, pi + 1.1102230246251565e-16, pi + 1.047197551196598],
 [pi - 1.3463968515384852,
  pi - 3.3306690738754696e-16,
  pi + 1.3463968515384876],
 [pi - 1.0471975511965999,
  pi + 1.1102230246251565e-16,
  pi + 1.3463968515384892],
 [pi - 0.448798950512829, pi + 0.44879895051282986, pi + 1.3463968515384954],
 [pi - 1.346396851538483, pi - 0.44879895051282787, pi + 1.0471975511965992],
 [pi - 1.3463968515384852, pi - 0.4487989505128315, pi + 0.4487989505128332],
 [pi - 1.0471975511966023, pi + 0.4487989505128334, pi + 1.3463968515385194]]

In [38]:
myChainFig.save("chain-figure.svg", figsize = 7)
##myChainFig.save("chain-figure-longer.svg", figsize = 7)